In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('df.csv')
print(df)

       stretch      stress  C10
0     1.043478    0.339335  1.3
1     1.086957    0.679834  1.3
2     1.130435    1.022495  1.3
3     1.173913    1.368172  1.3
4     1.217391    1.717594  1.3
5     1.260870    2.071390  1.3
6     1.304348    2.430107  1.3
7     1.347826    2.794219  1.3
8     1.391304    3.164142  1.3
9     1.434783    3.540242  1.3
10    1.478261    3.922840  1.3
11    1.521739    4.312223  1.3
12    1.565217    4.708643  1.3
13    1.608696    5.112328  1.3
14    1.652174    5.523480  1.3
15    1.695652    5.942281  1.3
16    1.739130    6.368894  1.3
17    1.782609    6.803467  1.3
18    1.826087    7.246134  1.3
19    1.869565    7.697015  1.3
20    1.913043    8.156221  1.3
21    1.956522    8.623852  1.3
22    2.000000    9.100000  1.3
23    2.043478    9.584748  1.3
24    2.086957   10.078174  1.3
25    2.130435   10.580348  1.3
26    2.173913   11.091335  1.3
27    2.217391   11.611194  1.3
28    2.260870   12.139981  1.3
29    2.304348   12.677747  1.3
30    2.

In [10]:
#Shuffle and Separate into test and train series by C10
train_inds, test_inds  = next(GroupShuffleSplit(test_size=0.1, n_splits=1, random_state = None).split(df, groups=df.C10))

#Get training data
df_training=df.iloc[train_inds, :]

#Get testing data
df_test=df.iloc[test_inds, :]

In [11]:
print(df_test)

       stretch      stress  C10
357   1.024691    0.148178  1.0
358   1.049383    0.296526  1.0
359   1.074074    0.445201  1.0
360   1.098765    0.594346  1.0
361   1.123457    0.744091  1.0
362   1.148148    0.894553  1.0
363   1.172840    1.045842  1.0
364   1.197531    1.198057  1.0
365   1.222222    1.351291  1.0
366   1.246914    1.505627  1.0
367   1.271605    1.661143  1.0
368   1.296296    1.817911  1.0
369   1.320988    1.975998  1.0
370   1.345679    2.135465  1.0
371   1.370370    2.296370  1.0
372   1.395062    2.458766  1.0
373   1.419753    2.622702  1.0
374   1.444444    2.788224  1.0
375   1.469136    2.955375  1.0
376   1.493827    3.124196  1.0
377   1.518519    3.294724  1.0
378   1.543210    3.466993  1.0
379   1.567901    3.641038  1.0
380   1.592593    3.816888  1.0
381   1.617284    3.994574  1.0
382   1.641975    4.174121  1.0
383   1.666667    4.355556  1.0
384   1.691358    4.538902  1.0
385   1.716049    4.724183  1.0
386   1.740741    4.911420  1.0
387   1.

,stretch,stress,C10
626,1.022472,0.364105,2.7
627,1.044944,0.728559,2.7
628,1.067416,1.093685,2.7
629,1.089888,1.459778,2.7
630,1.112360,1.827111,2.7
631,1.134831,2.195933,2.7
632,1.157303,2.566476,2.7
633,1.179775,2.938954,2.7
634,1.202247,3.313562,2.7
635,1.224719,3.690485,2.7
